# ⛽ Mexico Gas Prices Analysis
**Author:** Eduardo Torres <br>
**Date:** 2025-10-03 <br>
**Purpose:** Analyze gas prices in Mexico <br>

## Imports & Setup

In [2]:
import os
import glob
import requests
import pandas as pd
from lxml import etree
from datetime import datetime, date

## 📂 Data Loading
We handle three main datasets:

- `df_places`: Contains gas station information, including IDs, names, and geolocation (longitude & latitude).  
- `df_prices`: Contains gas prices for different fuel types at each station.  
- `df_historical`: Contains historical prices to track trends over time.

The data loading process consists of:

1. **Extracting XML files**: We read `places.xml` and `prices.xml` from our data source.  
2. **Parsing and converting to DataFrames**:  
   - `df_places` is created by extracting station details and geolocation.  
   - `df_prices` is created by extracting prices for each fuel type per station.  
3. **Sanity check**: We verify that every price entry has a corresponding station in `df_places`.  
4. **Merging**: We combine `df_places` and `df_prices` into `df_total` and append a `date` column.  
5. **Saving**: The combined dataset is saved as a compressed CSV with the current date.  

https://historico.datos.gob.mx/busca/dataset/estaciones-de-servicio-gasolineras-y-precios-finales-de-gasolina-y-diesel
https://www.gob.mx/cre/articulos/precios-vigentes-de-gasolinas-y-diesel https://historico.datos.gob.mx/busca/dataset/estaciones-de-servicio-gasolineras-y-precios-finales-de-gasolina-y-diesel

Opción final para github, descarga desde el código el xml y actualiza, solo la VPN no me deja en la del trabajo.

Opción de descarga directo desde el .xml

Gas station names and geolocalization

In [2]:
# Parse a .xml
tree = etree.parse("../data/places.xml")
root = tree.getroot()

# Extract data
data = []
for place in root.findall('place'):
    place_id = place.get('place_id')
    name = place.findtext('name')
    cre_id = place.findtext('cre_id')
    x = float(place.find('location/x').text)
    y = float(place.find('location/y').text)
    
    data.append({
        "place_id": place_id,
        "name": name,
        "cre_id": cre_id,
        "longitude": x,
        "latitude": y
    })

# Convert to DataFrame
df_places = pd.DataFrame(data)

df_places.head()

,place_id,name,cre_id,longitude,latitude
0,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.92140,32.47641
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,-99.74484,20.30370
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,-106.45140,31.71947
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,-103.30420,20.71413
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,-98.29977,26.03787


Gas station prices

In [3]:
# Parse a .xml
tree = etree.parse("../data/prices.xml")
root = tree.getroot()

# Extract data into a list of dicts
data = []
for place in root.findall('place'):
    place_id = place.get('place_id')
    for gas in place.findall('gas_price'):
        data.append({
            "place_id": place_id,
            "gas_type": gas.get('type'),
            "price": float(gas.text)
        })

# Convert to DataFrame
df_prices = pd.DataFrame(data)

df_prices.head()

,place_id,gas_type,price
0,11703,regular,22.95
1,11703,premium,23.89
2,11702,regular,24.50
3,11702,premium,27.30
4,11702,diesel,27.65


Sanity Check

In [4]:
# Check if there is some information losing when joining dfs
merged = df_places.merge(df_prices, on='place_id', how='outer', indicator=True)
right_only_count = merged['_merge'].value_counts().get('right_only', 0)
if right_only_count > 0:
    print(f"⚠☣❌Warning, there are {right_only_count} rows in df_prices that don't match with df_places❌☣⚠")
else: 
    print("✅ There's no information losing when joining dfs")


✅ There's no information losing when joining dfs


Merging places and prices

In [5]:
df_total = df_places.merge(df_prices,how='left', on='place_id')

df_total['date'] = date.today()

df_total.to_csv(
    f'../data/raw/gas_prices_{date.today().strftime("%Y%m%d")}.csv.gz', 
    index = False, 
    compression='gzip'
)

df_total.head()

,place_id,name,cre_id,longitude,latitude,gas_type,price,date
0,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.92140,32.47641,regular,22.69,2025-10-03
1,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.92140,32.47641,premium,26.99,2025-10-03
2,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.92140,32.47641,diesel,26.09,2025-10-03
3,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,-99.74484,20.30370,regular,23.99,2025-10-03
4,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,-99.74484,20.30370,premium,25.39,2025-10-03


<div class="alert alert-block alert-info">
<b>Note:</b> 'df_prices' and 'df_places' are already clean from source.
</div>

## 🧹 Data Cleaning

Historical Data

In [6]:
df_historical = pd.read_csv('../data/raw/gas_historical_prices.csv.gz',encoding='latin-1',compression='gzip')

df_historical = df_historical.rename(
    columns = {
        'NumeroPermiso':'cre_id',
        'SubProducto':'gas_type'
})

mapping = {
    'Regular': 'regular',
    'Premium': 'premium',
    'Diésel': 'diesel',
    'Diésel Automotríz':'diesel',
    'Diésel de Ultra Bajo Azufre (DUBA)':'diesel',
    'Diésel Agrícola/Marino':'diesel',
    'Diésel Industrial':'diesel'
}

df_historical['gas_type'] = df_historical['gas_type'].replace(mapping)

df_historical = df_historical.melt(id_vars=['cre_id','gas_type'], var_name='date',value_name='price')

df_historical['date'] = pd.to_datetime(df_historical['date'], format='%d/%m/%Y')

df_historical = df_historical[['cre_id','gas_type','price','date']]

df_historical = df_places.merge(df_historical,how='outer',on='cre_id',indicator=True)

del df_places

# Check if there is some information losing when joining dfs
both_count = df_historical['_merge'].value_counts().get('both', 0)
right_only_count = df_historical['_merge'].value_counts().get('right_only', 0)
left_only_count = df_historical['_merge'].value_counts().get('left_only', 0)
total_count = float(both_count + right_only_count + left_only_count)
print(f"Rows in Both Dataframes: {both_count}({both_count/total_count*100:.2f}%)")
print(f"Left Only: {left_only_count}({left_only_count/total_count*100:.2f}%)")
print(f"Right Only: {right_only_count}({right_only_count/total_count*100:.2f}%)")

df_historical = df_historical.loc[df_historical['_merge'].isin(['both'])]

df_historical.drop(columns=['_merge'], inplace=True)

df_historical

Rows in Both Dataframes: 120276330(99.97%)
Left Only: 713(0.00%)
Right Only: 31650(0.03%)


,place_id,name,cre_id,longitude,latitude,gas_type,price,date
0,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.9214,32.47641,diesel,NaN,2017-01-01
1,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.9214,32.47641,premium,NaN,2017-01-01
2,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.9214,32.47641,regular,NaN,2017-01-01
3,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.9214,32.47641,diesel,NaN,2017-01-02
4,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.9214,32.47641,premium,NaN,2017-01-02
...,...,...,...,...,...,...,...,...
120277037,30189,"PETRALL, S.A. DE C.V.",PL/26245/EXP/ES/2025,-102.3930,21.88831,premium,25.90,2025-08-30
120277038,30189,"PETRALL, S.A. DE C.V.",PL/26245/EXP/ES/2025,-102.3930,21.88831,regular,25.90,2025-08-30
120277039,30189,"PETRALL, S.A. DE C.V.",PL/26245/EXP/ES/2025,-102.3930,21.88831,diesel,23.95,2025-08-31
120277040,30189,"PETRALL, S.A. DE C.V.",PL/26245/EXP/ES/2025,-102.3930,21.88831,premium,25.90,2025-08-31


<div class="alert alert-block alert-warning">
<b>Warning:</b> There are .03% Missing Values due to df_places doesn't contain all historical info about gas stations location.
</div>

Union raw data datafames with gas_historical_prices

In [7]:
# Pattern to match files.
search_path  = "../data/raw/gas_prices*.csv.gz"

# Find all files matching the pattern
files_to_union = glob.glob(search_path)

# Load and concatenate all matching files
df_list = [pd.read_csv(f, compression='gzip') for f in files_to_union]
df_new = pd.concat(df_list, ignore_index=True)

del df_list

# Union with historical dataframe
df_union = pd.concat([df_historical, df_new], ignore_index=True)

del df_new

df_union

,place_id,name,cre_id,longitude,latitude,gas_type,price,date
0,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.9214,32.47641,diesel,NaN,2017-01-01 00:00:00
1,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.9214,32.47641,premium,NaN,2017-01-01 00:00:00
2,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.9214,32.47641,regular,NaN,2017-01-01 00:00:00
3,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.9214,32.47641,diesel,NaN,2017-01-02 00:00:00
4,2039,"ESTACION HIPODROMO, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.9214,32.47641,premium,NaN,2017-01-02 00:00:00
...,...,...,...,...,...,...,...,...
120348851,30188,"GAZPRO, S.A. DE C.V.",PL/26244/EXP/ES/2025,-100.4652,20.62897,NaN,NaN,2025-10-03
120348852,30189,"PETRALL, S.A. DE C.V.",PL/26245/EXP/ES/2025,-102.3930,21.88831,regular,23.98,2025-10-03
120348853,30189,"PETRALL, S.A. DE C.V.",PL/26245/EXP/ES/2025,-102.3930,21.88831,premium,26.05,2025-10-03
120348854,30189,"PETRALL, S.A. DE C.V.",PL/26245/EXP/ES/2025,-102.3930,21.88831,diesel,26.90,2025-10-03


In [3]:
df_final = pd.read_csv('../data/processed/mexico_gas_prices.csv.gz',compression='gzip')
df_final

EOFError: Compressed file ended before the end-of-stream marker was reached

<div class="alert alert-block alert-success">
<b>Up to you:</b> Use green boxes sparingly, and only for some specific 
purpose that the other boxes can't cover. For example, if you have a lot 
of related content to link to, maybe you decide to use green boxes for 
related links from each section of a notebook.
</div>

<div class="alert alert-block alert-danger">
<b>Just don't:</b> In general, avoid the red boxes. These should only be
used for actions that might cause data loss or another major issue.
</div>